## Internet Firewall Hit Counter ##

This demonstrates a DIY hit counter for Internet Firewall rules, using [Gradio](https://www.gradio.app/) to create the user interface. If Gradio not part of your existing Python environment you will need to install it using pip, together with all other prerequisite modules.

The hit counter uses two different Cato API calls:
1. [InternetFirewall policy](https://api.catonetworks.com/documentation/#query-policy.internetFirewall.policy) to retrieve a list of firewall rules.
2. [events](https://api.catonetworks.com/documentation/#query-events) to retrieve firewall rule hit counts.

There are issues with this method which can affect the accuracy of the result:
* There is a limit to the number of key:value pairs which will be fetched in a single events call, so if the number of firewall rules exceeds the limit, not all firewall rules will be fetched. Fortunately the limit at time of writing is 5000. It is unlikely that any single Cato account will have more than 5000 Internet Firewall rules.
* Only rules which have an action to log an event will be counted. Rules which don't log an event will still appear in the rules, but with a zero count.

### Preparation
First let's import the modules we need:

In [ ]:
import csv
import json
import os
import random
from collections import defaultdict

import gradio as gr
import pandas as pd

#
# Helper module 
# https://github.com/catonetworks/data-analytics/blob/main/notebooks/cato.py
#
from cato import API

### Making the first API call - InternetFirewall policy ###
We need a function which makes the call, and returns a list of Internet Firewall rule names and their position as a dictionary of position:rule name.

In [ ]:
def get_rules(C, ID):
	query = """query InternetFirewall($accountId: ID!) {
  policy(accountId: $accountId) {
	internetFirewall {
	  policy {
		rules {
		  rule {
			index
			name
		  }
		}
	  }
	}
  }
}"""
	variables = {
		"accountId":ID
	}
	success, rules = C.send("InternetFirewall", variables, query)
	return {rule["rule"]["index"]:rule["rule"]["name"] for rule in rules["data"]["policy"]["internetFirewall"]["policy"]["rules"]}

### Making the second API call - events ###

We need another function to make the events call which retrieves the Internet Firewall events by rule name:

In [ ]:
def get_hits(C, ID):
    query = """query events($accountID: ID!, $filters: [EventsFilter!]) {
  events(
	accountID: $accountID
	timeFrame: "last.P7D"
	filters: $filters
	dimensions: [{fieldName: rule_name}]
	measures: [{fieldName: event_count, aggType: sum}]
  ) {
	records(limit: 250) {
	  fieldsMap
	}
  }
}"""
    variables = {
		"accountID":ID,
		"filters": [{"fieldName": "event_sub_type", "operator": "is", "values": ["Internet Firewall"]}]
	}
    success, hits = C.send("events", variables, query)
    rule_hits = defaultdict(lambda:0) 
    for record in hits["data"]["events"]["records"]:
        rule_hits[record["fieldsMap"]["rule_name"]] = int(record["fieldsMap"]["event_count"])
    return rule_hits

### Combining the rules and hits into a DataFrame ###

The easiest way to publish the data will be as a table, and the easiest way to construct a table is to combine the rule data with the hits in a Pandas DataFrame:

In [ ]:
def create_dataframe(rules, hits):
    data = {
        "Position":[],
        "Rule":[],
        "Hits": [],
    }
    for index, rule_name in rules.items():
        data["Position"].append(index)
        data["Rule"].append(rule_name)
        data["Hits"].append(hits[rule_name])
    return pd.DataFrame(data)

### Putting it all together as a Gradio app ###
Finally we need a function to orchestrate the data pipeline, incorporate that as the click action for a Gradio button, and launch the app.

In [ ]:
def go(ID, key):
    #
    # Create the API connection
    #
    C = API(key)    
    #
    # Get the firewall rules
    #
    rules = get_rules(C,ID)
    #
    # Get the hits
    #
    hits = get_hits(C,ID)
    #
    # Create dataframe
    #
    return create_dataframe(rules, hits)


#
# Gradio app UI
#
with gr.Blocks() as demo:
    with gr.Column():
        input_id = gr.Textbox(label="Cato Account ID")
        input_key = gr.Textbox(label="Cato API Key", type="password")
        output = gr.Dataframe()
        button = gr.Button("Load Firewall Rules")
        button.click(go, inputs=[input_id, input_key], outputs=[output])
demo.launch()        